In [1]:
!pip install statsmodels


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import statsmodels.formula.api as smf


In [3]:
umsaetze_df = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv')
wetter_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/wetter.csv')
#wettercode_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/Wettercode.csv')
#feiertage_sh_ausgeschrieben_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/feiertage_sh_ausgeschrieben.csv')
kiwo_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/kiwo.csv')
schulferein_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/Variablen_CSV/schulferien.csv')
sonntage_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/Variablen_CSV/sonntage.csv')
#fremdenverkehr_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/Variablen_CSV/Fremdenverkehr.csv')
#passagieraufkommen_df = pd.read_csv('/workspaces/UmsatzWetterBackereiKiel/0_DataPreparation/Variablen_CSV/Passagieraufkommen.csv')



In [4]:
print(umsaetze_df.head)
print(wetter_df.head)
#print(wettercode_df.head)
#print(feiertage_sh_ausgeschrieben_df.head)
print(kiwo_df.head)
print(schulferein_df.head)
print(sonntage_df.head)
#print(fremdenverkehr_df.head)
#print(passagieraufkommen_df.head)

<bound method NDFrame.head of            Datum  Warengruppe      Umsatz
0     2013-07-01            1  148.828353
1     2013-07-02            1  159.793757
2     2013-07-03            1  111.885594
3     2013-07-04            1  168.864941
4     2013-07-05            1  171.280754
...          ...          ...         ...
9329  2017-12-21            6   87.471228
9330  2017-12-22            6   71.911652
9331  2017-12-23            6   84.062223
9332  2017-12-24            6   60.981969
9333  2017-12-27            6   34.972644

[9334 rows x 3 columns]>
<bound method NDFrame.head of            Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0     2012-01-01         8.0      9.8250                   14        58.0
1     2012-01-02         7.0      7.4375                   12         NaN
2     2012-01-03         8.0      5.5375                   18        63.0
3     2012-01-04         4.0      5.6875                   19        80.0
4     2012-01-05         6.0      5.3000

In [5]:
# Annahme: Das Datum ist im Format 'YYYY-MM-DD'
# Datum formatieren 

umsaetze_df['Datum'] = pd.to_datetime(umsaetze_df['Datum'])
wetter_df['Datum'] = pd.to_datetime(wetter_df['Datum'])
kiwo_df['Datum'] = pd.to_datetime(kiwo_df['Datum'])
schulferein_df['Datum'] = pd.to_datetime(schulferein_df['Datum'])
sonntage_df['Datum'] = pd.to_datetime(sonntage_df['Datum'])

#rohertrag_df['Datum'] = pd.to_datetime(rohertrag_df['Datum'])
#Fremdenverkehr_df['Datum'] = pd.to_datetime(Fremdenverkehr_df['Datum'])
#Passagiere_df['Datum'] = pd.to_datetime(Passagiere_df['Datum'])
#WetterB_df['Datum'] = pd.to_datetime(WetterB_df['Datum'])

In [6]:
# Daten zusammenführen
#df = pd.merge(umsaetze_df, wetter_df, kiwo_df, schulferein_df, sonntage_df, on='Datum')

# Annahme: Die DataFrames sind bereits definiert
# umsaetze_df, wetter_df, kiwo_df, schulferein_df, sonntage_df

# Schrittweise Zusammenführung der DataFrames basierend auf 'Datum'
merged_df = pd.merge(wetter_df, kiwo_df, on='Datum', how='outer')
merged_df = pd.merge(merged_df, umsaetze_df, on='Datum', how='outer')
merged_df = pd.merge(merged_df, schulferein_df, on='Datum', how='outer')
merged_df = pd.merge(merged_df, sonntage_df, on='Datum', how='outer')


# Sortieren nach Datum
merged_df = merged_df.sort_values('Datum')

# Ausgabe head
print(merged_df.head)


<bound method NDFrame.head of            Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0     2012-01-01         8.0      9.8250                 14.0        58.0   
1     2012-01-02         7.0      7.4375                 12.0         NaN   
2     2012-01-03         8.0      5.5375                 18.0        63.0   
3     2012-01-04         4.0      5.6875                 19.0        80.0   
4     2012-01-05         6.0      5.3000                 23.0        80.0   
...          ...         ...         ...                  ...         ...   
10128 2019-07-28         3.0     23.3500                 14.0         5.0   
10129 2019-07-29         6.0     25.2500                  7.0        61.0   
10130 2019-07-30         7.0     20.7375                  8.0        61.0   
10131 2019-07-31         6.0     20.4500                  7.0        61.0   
10132 2019-08-01         5.0     21.0625                  9.0        61.0   

       KielerWoche  Warengruppe  Umsatz  Feri

In [7]:
#na 0 muss raus

merged_df.fillna({"KielerWoche": 0, "Ferien": 0, "Verkaufsoffen": 0, "Wettercode": 0, "Warengruppe": 0, "Umsatz": 0}, inplace=True)
print(merged_df.head())

       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0 2012-01-01         8.0      9.8250                 14.0        58.0   
1 2012-01-02         7.0      7.4375                 12.0         0.0   
2 2012-01-03         8.0      5.5375                 18.0        63.0   
3 2012-01-04         4.0      5.6875                 19.0        80.0   
4 2012-01-05         6.0      5.3000                 23.0        80.0   

   KielerWoche  Warengruppe  Umsatz  Ferien  Verkaufsoffen  
0          0.0          0.0     0.0     0.0            0.0  
1          0.0          0.0     0.0     0.0            0.0  
2          0.0          0.0     0.0     0.0            0.0  
3          0.0          0.0     0.0     0.0            0.0  
4          0.0          0.0     0.0     0.0            0.0  


In [8]:
"""
# Start- und Enddatum definieren
start_date = '2012-06-03'
end_date = '2019-11-03'

# Datenreihe mit allen Daten im Zeitraum erstellen
all_dates = pd.date_range(start=start_date, end=end_date)

# DataFrame mit allen Daten erstellen
all_dates_merged = pd.DataFrame(all_dates, columns=['Datum'])

# Löschen aller Zeilen ab Datum '2019-08-01' und vor '2013-04-29'
datum_zum_loeschen = '2019-08-01'
datum_zum_loeschen_2 = '2013-04-29'
all_dates_merged = all_dates_merged[(all_dates_merged['Datum'] < datum_zum_loeschen) & (all_dates_merged['Datum'] > datum_zum_loeschen_2)]
print(all_dates_merged.head())
"""



"\n# Start- und Enddatum definieren\nstart_date = '2012-06-03'\nend_date = '2019-11-03'\n\n# Datenreihe mit allen Daten im Zeitraum erstellen\nall_dates = pd.date_range(start=start_date, end=end_date)\n\n# DataFrame mit allen Daten erstellen\nall_dates_merged = pd.DataFrame(all_dates, columns=['Datum'])\n\n# Löschen aller Zeilen ab Datum '2019-08-01' und vor '2013-04-29'\ndatum_zum_loeschen = '2019-08-01'\ndatum_zum_loeschen_2 = '2013-04-29'\nall_dates_merged = all_dates_merged[(all_dates_merged['Datum'] < datum_zum_loeschen) & (all_dates_merged['Datum'] > datum_zum_loeschen_2)]\nprint(all_dates_merged.head())\n"

In [9]:
# Split (Training und Validierung)

train_df = merged_df[(merged_df["Datum"] >= "01.07.2013") & (merged_df["Datum"] <= "31.07.2017")]
validation_df = merged_df[(merged_df["Datum"] >= "01.08.2017") & (merged_df["Datum"] <= "31.07.2018")]

print(f"Training Data Länge: {len(train_df)}")
print(f"Validation Data Länge: {len(validation_df)}")

print(train_df.head)
print(validation_df.head)

Training Data Länge: 7585
Validation Data Länge: 2862
<bound method NDFrame.head of           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
333  2013-04-30         3.0   11.116666                 15.0         0.0   
334  2013-05-01         0.0    9.900000                 11.0         0.0   
335  2013-05-02         1.0   11.037500                 12.0         0.0   
336  2013-05-03         0.0   12.912500                  9.0         0.0   
337  2013-05-04         0.0   17.950000                 18.0         0.0   
...         ...         ...         ...                  ...         ...   
7917 2017-07-31         2.0   22.750000                  9.0         0.0   
7916 2017-07-31         2.0   22.750000                  9.0         0.0   
7915 2017-07-31         2.0   22.750000                  9.0         0.0   
7914 2017-07-31         2.0   22.750000                  9.0         0.0   
7913 2017-07-31         2.0   22.750000                  9.0         0.0   

   

In [10]:
import statsmodels.formula.api as smf

# Fit the linear model
mod = smf.ols('Umsatz ~ KielerWoche + Verkaufsoffen + Ferien + C(Warengruppe) + Temperatur + Windgeschwindigkeit', data=merged_df).fit()

print(mod.summary())


                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     2595.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:35:25   Log-Likelihood:                -58141.
No. Observations:               10103   AIC:                         1.163e+05
Df Residuals:                   10091   BIC:                         1.164e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -46.98

In [13]:
import statsmodels.formula.api as smf

# Fit the linear model
mod = smf.ols('Umsatz ~ KielerWoche + Verkaufsoffen + Ferien + C(Warengruppe) + Temperatur:Windgeschwindigkeit', data=merged_df).fit()

print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     2778.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        18:42:08   Log-Likelihood:                -58243.
No. Observations:               10103   AIC:                         1.165e+05
Df Residuals:                   10092   BIC:                         1.166e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

**Zusammenfassung des Vergleiches zweier Modelle mit unterschiedlicher Einbeziehung der Temperatur** 

* Modell 1 passt die Daten etwas besser an als Modell 2, wie durch leicht höhere R²- und Adjusted R²-Werte.

* Beide Modelle zeigen, dass Warengruppen, Kieler Woche, verkaufsoffene Tage und Ferien signifikante Einflüsse auf den Umsatz haben. Modell 1 beinhaltet zusätzliche Variablen (Temperatur und Windgeschwindigkeit), wobei die Windgeschwindigkeit nicht signifikant ist, während Modell 2 einen signifikanten Interaktionsterm (Temperatur) beinhaltet.